## Model with Borough Included

In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [2]:
df = pq.read_table(source="/Users/danielapintoveizaga/Downloads/mlPreproc.parquet").to_pandas()

In [3]:
pd.set_option('display.max_columns', 500)
df

,created_date,borough,counts
0,2010-01-01,bronx,178
1,2010-01-01,brooklyn,182
2,2010-01-01,manhattan,158
3,2010-01-01,queens,143
4,2010-01-01,staten island,16
...,...,...,...
20213,2020-04-30,brooklyn,234
20214,2020-04-30,manhattan,173
20215,2020-04-30,queens,164
20216,2020-04-30,staten island,20


In [4]:
def festivo(date):
    '''
    Recorre el arreglo de dias festivos e indica si un dia es festivos
    Funcion auxiliar a create_feature_table
    '''
    import datetime
    h = ['2010-01-01', '2010-12-31', '2010-01-18', '2010-02-15', '2010-05-31', '2010-07-04', '2010-07-05', '2010-09-06', '2010-10-11', '2010-11-11', '2010-11-25', '2010-12-25', '2010-12-24', '2011-01-01', '2010-12-31', '2011-01-17', '2011-02-21', '2011-05-30', '2011-07-04', '2011-09-05', '2011-10-10', '2011-11-11', '2011-11-24', '2011-12-25', '2011-12-26', '2012-01-01', '2012-01-02', '2012-01-16', '2012-02-20', '2012-05-28', '2012-07-04', '2012-09-03', '2012-10-08', '2012-11-11', '2012-11-12', '2012-11-22', '2012-12-25', '2013-01-01', '2013-01-21', '2013-02-18', '2013-05-27', '2013-07-04', '2013-09-02', '2013-10-14', '2013-11-11', '2013-11-28', '2013-12-25', '2014-01-01', '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25', '2015-01-01', '2015-01-19', '2015-02-16', '2015-05-25', '2015-07-04', '2015-07-03', '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26', '2015-12-25', '2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', '2016-12-26', '2017-01-01', '2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29', '2017-07-04', '2017-09-04', '2017-10-09', '2017-11-11', '2017-11-10', '2017-11-23', '2017-12-25', '2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28', '2018-07-04', '2018-09-03', '2018-10-08', '2018-11-11', '2018-11-12', '2018-11-22', '2018-12-25', '2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27', '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11', '2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20', '2020-02-17', '2020-05-25', '2020-07-04', '2020-07-03', '2020-09-07', '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25']
    h=pd.to_datetime(h).date
    for festive in h:
        if(date==festive):
            return 1
    return 0

In [5]:
def flags(x):
    if(x<0):
        return 0
    else:
        return 1

In [6]:
df['created_date_year'] = pd.DatetimeIndex(df['created_date']).year
df['created_date_month'] = pd.DatetimeIndex(df['created_date']).month
df['created_date_day'] = pd.DatetimeIndex(df['created_date']).day

# Creamos la variable created_date_dow (numero del dia de la semana de la fecha created_date)
df['created_date_dow'] = pd.DatetimeIndex(df['created_date']).dayofweek

# Creamos la variable created_date_woy (numero de la semana del año de la fecha created_date)
df['created_date_woy'] = pd.DatetimeIndex(df['created_date']).week

# Creamos variable holiday
df["date_holiday"]=df["created_date"].apply(festivo)

history_days=10
#fecha de inicio
for i in range(1,history_days):
    for j in range(len(df["counts"])):
        if(j==0):
            var_name =  f"number_cases_{i}_days_ago"
            df[var_name]=0
        if(j<i):
            df[var_name][j]=0
        else:
            df[var_name][j]=df["counts"][(j-i)]

#variable respuesta
means=df.loc[:,['created_date_month','counts']]
means=means.groupby(['created_date_month'],as_index=False).mean()
means.columns=['created_date_month','month_mean']
df=pd.merge(df,means,how='left',on=["created_date_month"])

/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/danielapintoveizaga/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df

,created_date,borough,counts,created_date_year,created_date_month,created_date_day,created_date_dow,created_date_woy,date_holiday,number_cases_1_days_ago,number_cases_2_days_ago,number_cases_3_days_ago,number_cases_4_days_ago,number_cases_5_days_ago,number_cases_6_days_ago,number_cases_7_days_ago,number_cases_8_days_ago,number_cases_9_days_ago,month_mean
0,2010-01-01,bronx,178,2010,1,1,4,53,1,0,0,0,0,0,0,0,0,0,113.062914
1,2010-01-01,brooklyn,182,2010,1,1,4,53,1,178,0,0,0,0,0,0,0,0,113.062914
2,2010-01-01,manhattan,158,2010,1,1,4,53,1,182,178,0,0,0,0,0,0,0,113.062914
3,2010-01-01,queens,143,2010,1,1,4,53,1,158,182,178,0,0,0,0,0,0,113.062914
4,2010-01-01,staten island,16,2010,1,1,4,53,1,143,158,182,178,0,0,0,0,0,113.062914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213,2020-04-30,brooklyn,234,2020,4,30,3,18,0,192,22,157,233,283,196,22,216,217,146.180877
20214,2020-04-30,manhattan,173,2020,4,30,3,18,0,234,192,22,157,233,283,196,22,216,146.180877
20215,2020-04-30,queens,164,2020,4,30,3,18,0,173,234,192,22,157,233,283,196,22,146.180877
20216,2020-04-30,staten island,20,2020,4,30,3,18,0,164,173,234,192,22,157,233,283,196,146.180877


In [8]:
#flag de arriba o abajo del promedio
df["mean_flag"]= df["counts"] - df["month_mean"]
df["mean_flag"]=df["mean_flag"].apply(flags)


In [9]:
df

,created_date,borough,counts,created_date_year,created_date_month,created_date_day,created_date_dow,created_date_woy,date_holiday,number_cases_1_days_ago,number_cases_2_days_ago,number_cases_3_days_ago,number_cases_4_days_ago,number_cases_5_days_ago,number_cases_6_days_ago,number_cases_7_days_ago,number_cases_8_days_ago,number_cases_9_days_ago,month_mean,mean_flag
0,2010-01-01,bronx,178,2010,1,1,4,53,1,0,0,0,0,0,0,0,0,0,113.062914,1
1,2010-01-01,brooklyn,182,2010,1,1,4,53,1,178,0,0,0,0,0,0,0,0,113.062914,1
2,2010-01-01,manhattan,158,2010,1,1,4,53,1,182,178,0,0,0,0,0,0,0,113.062914,1
3,2010-01-01,queens,143,2010,1,1,4,53,1,158,182,178,0,0,0,0,0,0,113.062914,1
4,2010-01-01,staten island,16,2010,1,1,4,53,1,143,158,182,178,0,0,0,0,0,113.062914,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213,2020-04-30,brooklyn,234,2020,4,30,3,18,0,192,22,157,233,283,196,22,216,217,146.180877,1
20214,2020-04-30,manhattan,173,2020,4,30,3,18,0,234,192,22,157,233,283,196,22,216,146.180877,1
20215,2020-04-30,queens,164,2020,4,30,3,18,0,173,234,192,22,157,233,283,196,22,146.180877,1
20216,2020-04-30,staten island,20,2020,4,30,3,18,0,164,173,234,192,22,157,233,283,196,146.180877,0


In [13]:
y_test = df["mean_flag"]

In [15]:
import pickle
# Saving y_test
y_test.to_pickle("./y_test.pkl")

In [16]:

#one hot encodings
#year
dummies=pd.get_dummies(df["created_date_year"],prefix='y')
df=pd.concat([df,dummies],axis=1)

In [17]:
#month
dummies=pd.get_dummies(df["created_date_month"],prefix='m')
df=pd.concat([df,dummies],axis=1)

#day
dummies=pd.get_dummies(df["created_date_day"],prefix='d')
df=pd.concat([df,dummies],axis=1)

#day of week
dummies=pd.get_dummies(df["created_date_dow"],prefix='dow')
df=pd.concat([df,dummies],axis=1)

#week of year
dummies=pd.get_dummies(df["created_date_woy"],prefix='woy')
df=pd.concat([df,dummies],axis=1)


In [18]:
#borough
dummies=pd.get_dummies(df["borough"],prefix='borough')
df=pd.concat([df,dummies],axis=1)


In [19]:

#drop original cols
df=df.drop(["created_date_year","created_date_month","created_date_day","created_date_dow","created_date_woy", "borough"],axis=1)


In [20]:
df

,created_date,counts,date_holiday,number_cases_1_days_ago,number_cases_2_days_ago,number_cases_3_days_ago,number_cases_4_days_ago,number_cases_5_days_ago,number_cases_6_days_ago,number_cases_7_days_ago,number_cases_8_days_ago,number_cases_9_days_ago,month_mean,mean_flag,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,woy_1,woy_2,woy_3,woy_4,woy_5,woy_6,woy_7,woy_8,woy_9,woy_10,woy_11,woy_12,woy_13,woy_14,woy_15,woy_16,woy_17,woy_18,woy_19,woy_20,woy_21,woy_22,woy_23,woy_24,woy_25,woy_26,woy_27,woy_28,woy_29,woy_30,woy_31,woy_32,woy_33,woy_34,woy_35,woy_36,woy_37,woy_38,woy_39,woy_40,woy_41,woy_42,woy_43,woy_44,woy_45,woy_46,woy_47,woy_48,woy_49,woy_50,woy_51,woy_52,woy_53,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,borough_unspecified
0,2010-01-01,178,1,0,0,0,0,0,0,0,0,0,113.062914,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1,2010-01-01,182,1,178,0,0,0,0,0,0,0,0,113.062914,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,2010-01-01,158,1,182,178,0,0,0,0,0,0,0,113.062914,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,2010-01-01,143,1,158,182,178,0,0,0,0,0,0,113.062914,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,2010-01-01,16,1,143,158,182,178,0,0,0,0,0,113.062914,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213,2020-04-30,234,0,192,22,157,233,283,196,22,216,217,146.180877,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
20214,2020-04-30,173,0,234,192,22,157,233,283,196,22,216,146.180877,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
20215,2020-04-30,164,0,173,234,192,22,157,233,283,196,22,146.180877,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
20216,

In [21]:
len(df.columns)

134

In [27]:
df
y=df["mean_flag"]
df2=df.drop(columns=["mean_flag","created_date","counts", "month_mean"])

In [28]:
df2

,date_holiday,number_cases_1_days_ago,number_cases_2_days_ago,number_cases_3_days_ago,number_cases_4_days_ago,number_cases_5_days_ago,number_cases_6_days_ago,number_cases_7_days_ago,number_cases_8_days_ago,number_cases_9_days_ago,y_2010,y_2011,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_2020,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,woy_1,woy_2,woy_3,woy_4,woy_5,woy_6,woy_7,woy_8,woy_9,woy_10,woy_11,woy_12,woy_13,woy_14,woy_15,woy_16,woy_17,woy_18,woy_19,woy_20,woy_21,woy_22,woy_23,woy_24,woy_25,woy_26,woy_27,woy_28,woy_29,woy_30,woy_31,woy_32,woy_33,woy_34,woy_35,woy_36,woy_37,woy_38,woy_39,woy_40,woy_41,woy_42,woy_43,woy_44,woy_45,woy_46,woy_47,woy_48,woy_49,woy_50,woy_51,woy_52,woy_53,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,borough_unspecified
0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
1,1,178,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,1,182,178,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,1,158,182,178,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,1,143,158,182,178,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213,0,192,22,157,233,283,196,22,216,217,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
20214,0,234,192,22,157,233,283,196,22,216,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
20215,0,173,234,192,22,157,233,283,196,22,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
20216,0,164,173,234,192,22,157,233,283,196,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [29]:
df2.to_pickle("./X_test.pkl")

# Modelado

In [18]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [19]:
#primero separamos los datos de entrenamiento y prueba
x_train, x_test, y_train, y_test= train_test_split(df2,y,test_size=0.8)

In [20]:
#separamos los primeros 70% de los datos para entrenar
X_train = df2[:int(df2.shape[0]*0.7)].to_numpy()
X_test = df2[int(df2.shape[0]*0.7):].to_numpy()
y_train = y[:int(df2.shape[0]*0.7)].to_numpy()
y_test = y[int(df2.shape[0]*0.7):].to_numpy()

In [21]:
X_train.shape

(14152, 131)

In [22]:
#partimos los datos con temporal cv

tscv=TimeSeriesSplit(n_splits=5)

for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val=X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]

In [23]:
model=RandomForestClassifier(max_depth=10,criterion='gini',n_estimators=100,n_jobs=-1)
model.fit(X_tr,y_tr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [24]:
#PRedecimos sobre datos de validacion
preds=model.predict(X_val)

In [25]:
model.score(X_train,y_train)

0.9449547767100056

In [26]:
model.score(X_val,y_val)

0.9151823579304496

In [27]:
model.score(X_test,y_test)

0.8519617540389054

In [28]:
1-sum(abs(preds-y_val))/len(preds)

0.9151823579304496

In [29]:
#guarda pickle
import pickle
modelo_borough=pickle.dumps(model)

In [30]:
#load pickle
desde_pickle=pickle.loads(modelo_borough)

In [31]:
#usa el pickle
desde_pickle.score(X_test,y_test) 

0.8519617540389054